In [ ]:
import pandas as pd
import numpy as np
import scipy.special
import nbimporter
import util_func
import time
import tensorflow as tf
from itertools import product

In [ ]:
class FE_Data_Prep:
    def __init__(self, beta, FE):
        # Pull necessary components from input variables 
        # and save as either class objects or variables depending on use-case 
        self.beta = beta
        self.data_path = FE.data_path
        self.BC_mask = FE.BC_mask
        coordinates = FE.coord
        nodes = FE.connect
        
        # load FE data on snapshots and element deletion into dataframes
        snap = pd.read_csv(r"{path}_Snapshots.csv".format(path=self.data_path), index_col=0)
        status = pd.read_csv(r"{path}_Status.csv".format(path=self.data_path), index_col=0)
        
        # save the full set of snapshots into numpy array for handling
        self.y_full = snap.iloc[:, 4:].to_numpy()#[:,BC_mask]
        
        # Create numpy array of booleans denoting element deletion
        stat = ~status.iloc[:, 4:].to_numpy(dtype=bool)
        # Sum boolean array on axis 1 to determine total number of deleted elements at each snapshot
        stat_sum = np.sum(stat, axis=1)
        
        # Declare unit vecotr pointing to axial midline of tear in x-y plane
        unit_vec = np.array([1/np.sqrt(2), 1/np.sqrt(2)])
        
        # Initialize storage lists for indices of nodes on axial mid line both above and below mid-height
        mid_line_above = []
        mid_line_below = []
        # Iterate over coordinates to search for desired nodes
        for i in range(len(coordinates)):
            c = coordinates[i,:2] #coordinate vector in x-y plane
            unit_c = c/np.linalg.norm(c) # take unit vector of coordinate vector
            
            # check if coordinate unit vector is near identical to reference unit vector
            if np.linalg.norm(unit_c-unit_vec)<0.01:
                if FE.coord[i, 2] > 7:
                    mid_line_above.append(i)
                else:
                    mid_line_below.append(i)

            # Save nodes indices of coordinates on lateral edges of tear
            if np.linalg.norm(coordinates[i,:] - [3.81213, 3.38787, 7])<0.01:
                min_left = i
            if np.linalg.norm(coordinates[i,:] - [3.38787, 3.81213, 7])<0.01:
                min_right = i

        # sort indices into numpy array by distance from mid-height
        ind_above = np.argsort(coordinates[mid_line_above,2]-7)
        ind_below = np.argsort(coordinates[mid_line_below,2]-7)

        # Select coordinates in ordered array arranged by distance from mid-height
        coord_above = coordinates[np.array(mid_line_above)[ind_above]]
        coord_below = coordinates[np.array(mid_line_below)[ind_below]]

        # Reduce indices to inlcude only nodes on the midline which are in the middle of the geomtry's through-thickness
        nodes_above = []
        for i in range(3, len(coord_above)+1, 3):
            norm_set = [np.linalg.norm(coord_above[i-3:i][k]) for k in range(3)]
            nodes_above.append(ind_above[i-3:i][np.argwhere(norm_set == np.median(norm_set)).flatten()])
        nodes_below = []
        for i in range(3, len(coord_below)+1, 3):
            norm_set = [np.linalg.norm(coord_below[i-3:i][k]) for k in range(3)]
            nodes_below.append(ind_below[i-3:i][np.argwhere(norm_set == np.median(norm_set)).flatten()])

        # Finalize index list and make into numpy array
        maj_above = np.array(mid_line_above)[np.hstack(nodes_above)]
        maj_below = np.flip(np.array(mid_line_below)[np.hstack(nodes_below)])

        # Initialize counting operators and storage to gather geomtric infor about tear ellipse at each snapshot
        num_del = 0
        da_i = 0
        da_c = 0
        a_i = np.zeros((len(stat_sum), 1))
        a_c = np.zeros((len(stat_sum), 1))
        min_axis = np.zeros((len(stat_sum), 1))
        area = np.zeros((len(stat_sum), 1))
        perimeter = np.zeros((len(stat_sum), 1))
        nu_i = np.zeros((len(stat_sum), 1))
        theta_i = np.zeros((len(stat_sum), 1))
        nu_c = np.zeros((len(stat_sum), 1))
        theta_c = np.zeros((len(stat_sum), 1))
        
        # Iterate using array which contains total number of delted elements at each snapshot as length reference
        for i in range(len(stat_sum)):
            # Reshape snapshots to nodal displacement configuration
            node_disp = self.y_full[i,:].reshape((6225,3))
            # Compute coordinates of nodes in deformed mesh
            disp_field = FE.coord + node_disp

            # check if tear propagation has occured on current step
            if (stat_sum[i] != num_del and stat_sum[i] % 2 == 0):
                # find all elements which have been deleted
                curr_del = np.argwhere(stat[i,:]).flatten()
                # create array which determines whether said elements are above or below mid-height
                temp = np.array([coordinates[nodes[k][0]][2] > 7 for k in curr_del])
                
                # update counting variables to reflect how many elements are delted above and below mid-height
                da_i = np.sum(temp)
                da_c = np.sum([not k for k in temp])
                
                # update total number of deleted elements
                num_del = stat_sum[i]

            # declare node indices corresponding to upper and lower ends of tear major axis
            maj_ind_a = maj_above[int(da_i/2)]
            maj_ind_b = maj_below[int(da_c/2)]

            # compute vector which points to mid-height between lateral edges of tear
            mid_height = (disp_field[min_left] + disp_field[min_right])/2

            # compute upper and lower major axes of tear relative to mid-height
            a_i[i] = np.abs(disp_field[maj_ind_a][2] - mid_height[2])
            a_c[i] = np.abs(mid_height[2] - disp_field[maj_ind_b][2])
            
            # compute minor axis of tear
            min_axis[i] = np.linalg.norm(disp_field[min_left] - disp_field[min_right])/2

            # compute area of approximate ellipse projection of tear
            area[i] = (np.pi/2)*(a_i[i]*min_axis[i] + a_c[i]*min_axis[i])

            # numerically approximate perimeter of ellipse projection of tear using infinite sum taken to 10 iterations
            h1 = (a_i[i] - min_axis[i])**2 / (a_i[i] + min_axis[i])**2
            h2 = (a_c[i] - min_axis[i])**2 / (a_c[i] + min_axis[i])**2
            perimeter[i] = (np.pi*(a_i[i] + min_axis[i])*np.sum([(h1**n)*scipy.special.binom(0.5, n) for n in range(10)])
                            + np.pi*(a_c[i] + min_axis[i])*np.sum([(h2**n)*scipy.special.binom(0.5, n) for n in range(10)]))/2

            # calculate angles between vectors pointing from origin to upper and lower ends of major axis 
            # and vector pointing to left end of minor axis
            theta_i[i] = (180/np.pi)*np.arccos( np.dot(disp_field[min_left], disp_field[maj_ind_a])/
                                   (np.linalg.norm(disp_field[min_left])*np.linalg.norm(disp_field[maj_ind_a])) )
            theta_c[i] = (180/np.pi)*np.arccos( np.dot(disp_field[min_left], disp_field[maj_ind_b])/
                                   (np.linalg.norm(disp_field[min_left])*np.linalg.norm(disp_field[maj_ind_b])) )
            
            # calculate angles vectors pointing from left end of minor axis to upper and lower ends of major axis
            # and vector pointing from left end of minor axis to right end of minor axis
            nu_i[i] = (180/np.pi)*np.arccos( np.dot(disp_field[min_left] - disp_field[maj_ind_a], disp_field[min_left] - disp_field[min_right])/
                                   (np.linalg.norm(disp_field[min_left] - disp_field[maj_ind_a])*np.linalg.norm(disp_field[min_left] - disp_field[min_right])) )
            nu_c[i] = (180/np.pi)*np.arccos( np.dot(disp_field[min_left] - disp_field[maj_ind_b], disp_field[min_left] - disp_field[min_right])/
                                   (np.linalg.norm(disp_field[min_left] - disp_field[maj_ind_b])*np.linalg.norm(disp_field[min_left] - disp_field[min_right])) )
        

        # Construct full set of ML inputs from FE parameters and computed geometric data
        self.x_full = np.hstack((snap.iloc[:,:3].to_numpy(), np.expand_dims(snap.iloc[:,3].to_numpy()*50, axis=1), 
                                a_i, a_c, min_axis, area, perimeter, theta_i, theta_c, nu_i, nu_c))
        
        
        # Initialize storage for reduced sample sets from full data 
        samples_x = []
        samples_y = []
        # create counting variables to separate by beta parameter case
        x_time = snap.iloc[:,3].to_numpy()
        start_ind = np.sort(np.argsort(x_time)[:8])
        stop_ind = start_ind - 1
        stop_ind = np.hstack([stop_ind[1:], stop_ind[0]])
        # iterate over array of counting varibales which separates beta parameter cases
        for i in range(len(stop_ind)):
            # select time stamps and subsets of x and y data from each beta parameter case
            times_mu = x_time[start_ind[i]:stop_ind[i]]
            x_mu = self.x_full[start_ind[i]:stop_ind[i],:]
            y_mu = self.y_full[start_ind[i]:stop_ind[i],:]

            # search for indices corresponding to approximately equal time steps of 0.01s (0.5 kPa)
            down_sample = [np.argmin(np.abs(times_mu-k)) for k in [x/100 for x in range(1,101)]]
            # ensure index uniqueness and combine into numpy array
            down_sample = np.unique(np.hstack(down_sample))
            
            # add samples from subsets to sample list
            samples_x.append(x_mu[down_sample,:])
            samples_y.append(y_mu[down_sample,:])
        
        # convert complete sample list for all beta parameter cases into numpy array and save as class object
        self.x = np.vstack(samples_x)
        self.y = np.vstack(samples_y)
        
        
        # Divide degrees of freedom into 2 subdomains using cross products to 
        # search for dofs inside or outside certatin angle sections
        a_o = [4.71, 1.95] # left outer bound to define subdomain near tear
        a_i = [4.29, 2.71] # left inner bound to define subdomain away from tear

        b_o = a_o[::-1] # right outer bound
        b_i = a_i[::-1] # right inner bound

        # define cross products of inner and outer bounds against which to compare other vectors
        A_ixB_i = (a_i[0]*b_i[1] - a_i[1]*b_i[0]) # always positive
        A_oxB_o = (a_o[0]*b_o[1] - a_o[1]*b_o[0]) # always positive

        # iterate over all coordinates to find which coordinate vectors belong to each subdomain
        ind_away_from_tear = []
        ind_near_tear = []
        for i in range(len(coordinates)):
            c = coordinates[i,:2] # coordinate vector in x-y plane

            # coordinate vector cross products
            A_ixC = (a_i[0]*c[1] - a_i[1]*c[0])
            CxB_i = (c[0]*b_i[1] - c[1]*b_i[0])
            CxA_i = (c[0]*a_i[1] - c[1]*a_i[0])

            A_oxC = (a_o[0]*c[1] - a_o[1]*c[0])
            CxB_o = (c[0]*b_o[1] - c[1]*b_o[0])
            CxA_o = (c[0]*a_o[1] - c[1]*a_o[0])

            # conditional to catch vectors outside of inner bounds
            if not ((CxB_i * CxA_i) <= 0 and (A_ixB_i * A_ixC)>=0):
                ind_away_from_tear.append(i)

            # conditional to catch vectors inside of outer bounds
            if ((CxB_o * CxA_o) <= 0 and (A_oxB_o * A_oxC) >=0):
                ind_near_tear.append(i)

        # iterate over connectivity matrix to determine which whole elements belong to each domain
        nodes_away = []
        nodes_near = []
        for i in range(len(nodes)):
            # search for nodes which contain indices retrieved via cross product method above
            found_a = np.isin(nodes[i,:], ind_away_from_tear)
            found_n = np.isin(nodes[i,:], ind_near_tear)
            
            # flag elements where all nodes reside in the subdomain bounds
            flag_a = np.sum(found_a) == 8
            flag_n = np.sum(found_n) == 8

            # add elements to subdomain set according to flags
            if (flag_a & ~flag_n):
                nodes_away.append(nodes[i,:])

            elif (flag_n & ~flag_a):
                nodes_near.append(nodes[i,:])

            elif (flag_a & flag_n):
                nodes_away.append(nodes[i,:])
                nodes_near.append(nodes[i,:])

        # combine subdomain nodes into numpy arrays
        nodes_away = np.vstack(nodes_away)
        nodes_near = np.vstack(nodes_near)

        # convert from node sets to dof sets for each subdomain
        dof_away = np.array([[3*x, 3*x+1, 3*x+2] for x in np.unique(nodes_away)]).flatten()
        dof_near = np.array([[3*x, 3*x+1, 3*x+2] for x in np.unique(nodes_near)]).flatten()

        # nodes_schwarz1 = (np.sum(np.isin(nodes_away, nodes_near), axis=1) == nodes_away.shape[1]).flatten()
        # nodes_schwarz2 = (np.sum(np.isin(nodes_near, nodes_away), axis=1) == nodes_away.shape[1]).flatten()

        # save subdomain dof sets for later use
        self.y_omega1 = self.y[:, dof_away]
        self.y_omega2 = self.y[:, dof_near]
        
        # save boolean masks which identify dofs that lie on the overlap region in each subdomain
        self.schwarz_mask1 = np.isin(dof_away, dof_near)
        self.schwarz_mask2 = np.isin(dof_near, dof_away) 
            

In [ ]:
#             stat_mu = ~status_subset.iloc[:, 4:].to_numpy(dtype=bool)
#             stat_sum = np.sum(stat_mu, axis=1)
#             val = np.unique(stat_sum)
#             val = val[val % 2 == 0][1:]

#             if enrich:
#                 prop_snap = [np.min(np.where(stat_sum==j)) for j in val]
#                 times_enrich = status_subset['t'].iloc[prop_snap].to_numpy()

#                 for c in range(len(times_enrich)):
#                     start = np.argmin(np.abs(times_mu-(times_enrich[c]-0.005)))
#                     end = np.argmin(np.abs(times_mu-(times_enrich[c]+0.005)))
#                     step = int(np.round((end-start)/30))
#                     down_sample.append(np.array(range(start,end,step)))


#     def get_POD_basis(self, A):
#         covariance = np.matmul(np.transpose(A),A)

#         L, V = np.linalg.eig(covariance)
#         Lambda = np.sort(np.real(L))[::-1]
#         L_arg = np.argsort(np.real(L))[::-1]

#         phi = np.real(V)[:, L_arg]

#         sigma = np.sqrt(Lambda)
#         sigma = sigma[~np.isnan(sigma)]
#     #     sigma = tf.where(tf.math.is_nan(sigma), tf.zeros_like(sigma), sigma)
#         sigma_inv = np.diag(sigma**-1)

#         psi = np.matmul(np.matmul(A, phi[:, :sigma.shape[0]]), sigma_inv)

#         return sigma, psi
    
     
#     def get_K_red_0(self, psi, l):
        
#         combination = self.beta.shape[1]**self.beta.shape[0]
#         K_red_0 = [[None]*combination for i in range(l)]

#         for j, mu in enumerate(product(self.beta[0], self.beta[1], self.beta[2])):
#             Kc_0 = cp.asarray(pd.read_csv(r"{path}_Kc0_{b_d}_{b_m}_{b_p}.csv"
#                                 .format(path=self.data_path,b_d=mu[0],b_m=mu[1],b_p=mu[2]), index_col=0).to_numpy())

#             print("checkpoint {ch}".format(ch=j))

#             for i in range(l):
#                 psi_l = psi[:,:i+1]
#                 K_red_0[i][j] = cp.matmul(cp.matmul(cp.transpose(psi_l),Kc_0), psi_l)
                
#         return K_red_0

#         self.beta_mu = []
#         self.times_mu = []
#         self.Uc_mu = []
#         self.Fc_mu = []
#         for i, mu in enumerate(product(beta[0], beta[1], beta[2])):
#             snap_subset = snap[(snap['beta_d'] == mu[0]) & (snap['beta_m'] == mu[1]) & (snap['beta_p'] == mu[2])]
#             #status_subset = status[(status['beta_d'] == mu[0]) & (status['beta_m'] == mu[1]) & (status['beta_p'] == mu[2])]
            
#             self.beta_mu.append(snap_subset.iloc[:, :3].to_numpy())
            
#             self.times_mu.append(snap_subset.iloc[:, 3].to_numpy())

#             self.Uc_mu.append(snap_subset.iloc[:, 4:].to_numpy().T[BC_mask,:])

#             self.Fc_mu.append(pd.read_csv(r"{path}_Fc_{b_d}_{b_m}_{b_p}.csv"
#                                        .format(path=data_path,b_d=mu[0],b_m=mu[1],b_p=mu[2]), index_col=0).to_numpy())